In [138]:
import requests
import os
import pandas as pd
import numpy as np
import time
import requests
import math
import datetime
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from DATASCI210_Functions import column_statitical_review,create_segments_from_dataframe_column,CreateActivityByDATE,fetch_transaction_details,CreateStandardizedDefinitions,ImportFiles,PoolActivitybyUser,SummarizeAllTransactions,GetHistoricalTradeActivity,UserSegmentCreation,Supplemental_Dictionary,CreateFinalDataSet,hex_to_int,AggregatePoolCalculations,RollingWindow,Data_Dict_Missing_Cols,TransactionUniqueinDataset,CummulativeSum,CummulativeCount,NEW_VARIABLE_CREATE

data_dictionary = pd.read_excel('data_dictionary_V2.xlsx')

pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', None)

# Upload Supplemental Files

# Import Supplemental Trade information
supplemental1 = pd.read_csv("FINAL_SUPPLEMENTAL_DATASET.csv")
supplemental2 = pd.read_csv("INITIAL_SUPPLEMENTAL_DATASET.csv")

supplemental_trade_info = pd.concat([supplemental1,supplemental2])

og_trade_df = ImportFiles('arbi','Arbitrage_Files')

# Remove Blanks
og_trade_df = og_trade_df[og_trade_df['p1.sender'].notnull()].copy()
og_trade_df = og_trade_df[og_trade_df['p0.sender'].notnull()].copy()
og_trade_df = og_trade_df[og_trade_df['p0.transaction_id'].notnull()].copy()

og_trade_df['p1.sender'] =    og_trade_df['p1.sender'].apply(lambda x: hex_to_int(x))
og_trade_df['p1.recipient'] = og_trade_df['p1.recipient'].apply(lambda x:  hex_to_int(x))
og_trade_df['p0.sender'] =    og_trade_df['p0.sender'].apply(lambda x: hex_to_int(x))
og_trade_df['p0.recipient'] = og_trade_df['p0.recipient'].apply(lambda x:  hex_to_int(x))
og_trade_df['time'] = og_trade_df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)))
og_trade_df['datetime'] = pd.to_datetime(og_trade_df['time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
og_trade_df['hour'] = og_trade_df['datetime'].dt.hour

try:
    og_trade_df.drop('Unnamed: 0',axis=1,inplace=True)
except:
    pass
    
final_df = CreateFinalDataSet(og_trade_df,supplemental1,supplemental2)

# Generate additional Records to link on Tran ID
#t = pd.concat([final_df[['p1.transaction_id']].drop_duplicates().rename(columns={'p1.transaction_id':'Transaction_ID'}),final_df[['p0.transaction_id']].drop_duplicates().rename(columns={'p0.transaction_id':'Transaction_ID'})]).drop_duplicates().merge(supplemental1[['Transaction_ID']],on='Transaction_ID',how='left',indicator=True)
#t[t['_merge']=='left_only'].drop_duplicates()['Transaction_ID'].to_csv('missing_tran_id',index=False)

# Generate final Dataset
final_df['DATE'] = final_df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)).replace(hour=0, minute=0, second=0, microsecond=0))
final_df['p0.t0_amount_abs'] = final_df['p0.t0_amount'].apply(lambda x:abs(x))
final_df['p1.t0_amount_abs']= final_df['p1.t0_amount'].apply(lambda x:abs(x))
final_df['COUNT'] = 1
pd.set_option('display.expand_frame_repr', False)

d = NEW_VARIABLE_CREATE(final_df)

d.head()


/var/folders/4m/j9kyjvln7ps4dkntd048fklh0000gn/T/ipykernel_42198/3219793041.py:24: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  supplemental1 = pd.read_csv("FINAL_SUPPLEMENTAL_DATASET.csv")
/var/folders/4m/j9kyjvln7ps4dkntd048fklh0000gn/T/ipykernel_42198/3219793041.py:25: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  supplemental2 = pd.read_csv("INITIAL_SUPPLEMENTAL_DATASET.csv")
/Users/derekdewald/DATASCI210/final_data_jun17/DATASCI210_Functions.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df,pd.read_csv(report)])
/Users/derekdewald/DATASCI210/final_data_jun17/DATASCI210_Functions.py:28: Dty

To Number of Transactions Matching on Transaction ID 454668
Number of Transaction Missing Link of Transaction ID 1553587
Numher of Transactions which are still Missing to Map to Transaction1: 263
Numher of Unique Transactions which are still Missing to Map to Transaction1: 255
Numher of Transactions which are still Missing to Map to Transaction2: 20102
Numher of Unique Transactions which are still Missing to Map to Transaction1: 774


,time,timestamp,bitcoin_trend,crypto_currency_trend,ethereum_trend,USDC_trend,WETH_trend,p1.transaction_time,p1.transaction_epoch_time,p1.t0_amount,p1.t1_amount,p1.t0_token,p1.t1_token,p1.tick,p1.sqrtPriceX96,p1.gasUsed,p1.gasPrice,p1.blockNumber,p1.sender,p1.recipient,p1.transaction_id,p1.transaction_type,p1.transaction_rate,p1.eth_price_usd,p1.transaction_fees_usd,p1.gas_fees_usd,p1.total_fees_usd,p0.transaction_time,p0.transaction_epoch_time,p0.t0_amount,p0.t1_amount,p0.t0_token,p0.t1_token,p0.tick,p0.sqrtPriceX96,p0.gasUsed,p0.gasPrice,p0.blockNumber,p0.sender,p0.recipient,p0.transaction_id,p0.transaction_type,p0.transaction_rate,p0.eth_price_usd,p0.transaction_fees_usd,p0.gas_fees_usd,p0.total_fees_usd,percent_change,total_gas_fees_usd,total_transaction_rate,total_transaction_fees_used,total_fees_usd,swap_go_nogo,global_eth_price_usd,datetime,hour,blockHash_trade1,blockNumber_trade1,from_trade1,gas_trade1,gasPrice_trade1,maxFeePerGas_trade1,maxPriorityFeePerGas_trade1,hash_trade1,nonce_trade1,to_trade1,transactionIndex_trade1,value_trade1,type_trade1,accessList_trade1,chainId_trade1,v_trade1,r_trade1,s_trade1,yParity_trade1,blockHash_trade2,blockNumber_trade2,from_trade2,gas_trade2,gasPrice_trade2,maxFeePerGas_trade2,maxPriorityFeePerGas_trade2,hash_trade2,nonce_trade2,to_trade2,transactionIndex_trade2,value_trade2,type_trade2,accessList_trade2,chainId_trade2,v_trade2,r_trade2,s_trade2,yParity_trade2,Transaction_ID_trade1,Transaction_ID_trade2,DATE,p0.t0_amount_abs,p1.t0_amount_abs,COUNT,TRADE_POOL1,TRADE_POOL2,TRADE_POOL_1_SELL_FLAG,TRADE_POOL_2_SELL_FLAG,TRADER_POOL1,TRADER_POOL2,SMART_CONTRACT_POOL1,SMART_CONTRACT_POOL2,Tran1Unique,Tran2Unique,p1.t0_amount_unique,p0.t0_amount_unique,BUY_SELL_BINARY1,BUY_SELL_BINARY2,BUY_SELL_BINARY1_unique,BUY_SELL_BINARY2_unique,POOL_1_HISTORICAL_VOL,POOL_2_HISTORICAL_VOL,POOL_1_HISTORICAL_TRADE_VAL,POOL_2_HISTORICAL_TRADE_VAL,POOL_1_DAILY_VOL,POOL_2_DAILY_VOL,POOL_1_DAILY_VAL,POOL_2_DAILY_VAL,TRADER_POOL1_DAILY_VOL,TRADER_POOL2_DAILY_VOL,TRADER_POOL1_DAILY_VAL,TRADER_POOL2_DAILY_VAL,TRADER_POOL1_HISTORICAL_VOL,TRADER_POOL2_HISTORICAL_VOL,TRADER_POOL1_HISTORICAL_VAL,TRADER_POOL2_HISTORICAL_VAL,SMART_CONTRACT_POOL_1_HISTORICAL_VOL,SMART_CONTRACT_POOL_2_HISTORICAL_VOL,SMART_CONTRACT_POOL_1_HISTORICAL_VAL,SMART_CONTRACT_POOL_2_HISTORICAL_VAL,SMART_CONTRACT_POOL_1_DAILY_VOL,SMART_CONTRACT_POOL_2_DAILY_VOL,SMART_CONTRACT_POOL_1_DAILY_VAL,SMART_CONTRACT_POOL_2_DAILY_VAL,POOL_1_HISTORICAL_TRANSACTION_POSITION,POOL_2_HISTORICAL_TRANSACTION_POSITION,POOL_1_HISTORICAL_POSITION_VALUE,POOL_2_HISTORICAL_POSITION_VALUE,POOL_1_DAILY_VOL_POSITION,POOL_2_DAILY_VOL_POSITION,POOL_1_DAILY_VAL_POSITION,POOL_2_DAILY_VAL_POSITION,target,RECORD_COUNT,TRADER1_60M_net_val,TRADER1_60M_total_vol,TRADER1_60M_net_position,TRADER2_60M_net_val,TRADER2_60M_total_vol,TRADER2_60M_net_position,SMART_CONTRACT_POOL1_60M_net_val,SMART_CONTRACT_POOL1_60M_total_vol,SMART_CONTRACT_POOL1_60M_net_position,SMART_CONTRACT_POOL2_60M_net_val,SMART_CONTRACT_POOL2_60M_total_vol,SMART_CONTRACT_POOL2_60M_net_position
0,2023-06-11 17:12:23,1686528743,30.00,51.00,27.00,37.00,49.00,2023-06-11 20:12:23,1686528743.00,-13512.56,7.72,USD Coin,Wrapped Ether,201634.00,1892786412730650516986981802573824.00,1158867.00,15204713385.00,17460342.00,597733001430176024049334587394716596403930790981,597733001430176024049334587394716596403930790981,0xa67c5f3cd4dddd9c28b9cc00ce6fe4d5187049a3345f...,0.00,0.00,1752.09,6.76,30.87,37.63,2023-06-11 20:12:23,1686528743.00,-8979.68,5.14,USD Coin,Wrapped Ether,201641.00,1893451369374087284701913788973056.00,1158867.00,15204713385.00,17460342.00,597733001430176024049334587394716596403930790981,597733001430176024049334587394716596403930790981,0xa67c5f3cd4dddd9c28b9cc00ce6fe4d5187049a3345f...,0.00,0.00,1750.86,27.02,30.85,57.87,-0.00,61.72,0.00,33.78,95.50,False,1749.37,2023-06-11 17:12:23,17,4747495228788291979487608950781561035723030343...,NaN,1142032312917384199936240033806992302955186595417,1158867.00,1520471338

## Test Whether Columns Are Missing From Data Dictionary

In [139]:
DATA_DICT_MISSING_COLS(d,data_dictionary)

Number of Missing Records:0


,FIELD_NAME,DESCRIPTION,EXPLANATION,EDA Observation,DATA_REVIEW,ML_MODEL,VariableType,DataType,Standardize,source,ReferencePool
0,time,Time of Transaction Start,NaN,NaN,NaN,NaN,Transaction,Time,NaN,NaN,NaN
1,timestamp,Timestamp of Transaction,NaN,NaN,NaN,NaN,Transaction,Time,NaN,NaN,NaN
2,bitcoin_trend,Value of rolling 7-day google search trend pop...,"Bitcoin, like Ethereum is a digital currency, ...","Overall the trend is linear upwards trends, su...",1.00,1.00,Sentiment,Float,NaN,https://trends.google.com/trends/,NaN
3,crypto_currency_trend,Value of rolling 7-day google search trend pop...,"Crypto Currency is a generalized term, which p...","Overall the trend is linear upwards trends, su...",1.00,1.00,Sentiment,Float,NaN,https://trends.google.com/trends/,NaN
4,ethereum_trend,Value of rolling 7-day google search trend pop...,"Ethereum is a digital currency, leveraging the...","Overall the trend is linear upwards, with incr...",1.00,1.00,Sentiment,Float,NaN,https://trends.google.com/trends/,NaN
...,...,...,...,...,...,...,...,...,...,...,...
157,SMART_CONTRACT_POOL1_60M_total_vol,TBD,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN
158,SMART_CONTRACT_POOL1_60M_net_position,TBD,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN
159,SMART_CONTRACT_POOL2_60M_net_val,TBD,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN
160,SMART_CONTRACT_POOL2_60M_total_vol,TBD,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN


### Export Monthly Files for purposes of iterative Training and reducing computation effort

In [ ]:
months = [datetime.datetime(2023,6,30),datetime.datetime(2023,7,31),datetime.datetime(2023,8,31),datetime.datetime(2024,9,30),
          datetime.datetime(2024,10,31),datetime.datetime(2024,11,30),datetime.datetime(2024,12,31),datetime.datetime(2024,1,31),
          datetime.datetime(2024,2,29),datetime.datetime(2024,3,31),datetime.datetime(2024,4,30),datetime.datetime(2024,5,31),
          datetime.datetime(2024,6,30),datetime.datetime(2024,7,30)]

for count,i in enumerate(months):
    if count == 0:
        d[d['time']<i].to_csv(f"Final Dataset//final_df_{i.strftime('%b-%y')}.csv",index=False)
    elif count == len(months)-1:
        pass
    else:
        d[(d['time']>months[count-1])&(d['time']<=months[count])].to_csv(f"Final Dataset//final_df_{i.strftime('%b-%y')}.csv",index=False)
          